# 🔢 MNIST Digit Classification with CNN

**Project**: Deep Learning - Computer Vision  
**Level**: Advanced  
**Dataset**: MNIST Handwritten Digits  

## 📋 Project Overview

This project classifies handwritten digits using Convolutional Neural Networks (CNN). We'll learn:

- Deep learning fundamentals
- CNN architecture design
- Convolution and pooling operations
- Data augmentation techniques
- Model regularization and optimization

Let's build our first deep learning model! 🧠

## 1. Import Libraries

In [ ]:
# Data manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Metrics and utilities
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print(f"🧠 TensorFlow version: {tf.__version__}")
print(f"🔥 GPU available: {tf.config.list_physical_devices('GPU')}")

## 2. Data Loading and Exploration

In [ ]:
# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print("🔢 MNIST dataset loaded!")
print(f"Training images: {X_train.shape}")
print(f"Training labels: {y_train.shape}")
print(f"Test images: {X_test.shape}")
print(f"Test labels: {y_test.shape}")
print(f"Image shape: {X_train[0].shape}")
print(f"Pixel value range: {X_train.min()} - {X_train.max()}")
print(f"Number of classes: {len(np.unique(y_train))}")

In [ ]:
# Visualize sample digits
fig, axes = plt.subplots(2, 5, figsize=(15, 8))
fig.suptitle('🔍 Sample MNIST Digits', fontsize=16, fontweight='bold')

for i in range(10):
    row, col = i // 5, i % 5
    
    # Find first occurrence of digit i
    idx = np.where(y_train == i)[0][0]
    
    axes[row, col].imshow(X_train[idx], cmap='gray')
    axes[row, col].set_title(f'Digit: {i}', fontweight='bold')
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Class distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Training set distribution
train_counts = np.bincount(y_train)
axes[0].bar(range(10), train_counts, color='skyblue', alpha=0.7)
axes[0].set_title('📊 Training Set - Digit Distribution', fontweight='bold', fontsize=14)
axes[0].set_xlabel('Digit')
axes[0].set_ylabel('Count')
axes[0].set_xticks(range(10))

# Add value labels
for i, count in enumerate(train_counts):
    axes[0].text(i, count + 50, str(count), ha='center', va='bottom', fontweight='bold')

# Test set distribution
test_counts = np.bincount(y_test)
axes[1].bar(range(10), test_counts, color='lightcoral', alpha=0.7)
axes[1].set_title('📊 Test Set - Digit Distribution', fontweight='bold', fontsize=14)
axes[1].set_xlabel('Digit')
axes[1].set_ylabel('Count')
axes[1].set_xticks(range(10))

# Add value labels
for i, count in enumerate(test_counts):
    axes[1].text(i, count + 20, str(count), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("📊 Dataset is well balanced across all digits!")

## 3. Data Preprocessing

In [ ]:
# Reshape and normalize the data
print("🔧 Preprocessing data...")

# Reshape to add channel dimension (for CNN)
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

# Normalize pixel values to [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Convert labels to categorical (one-hot encoding)
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

print(f"✅ Preprocessed training data shape: {X_train.shape}")
print(f"✅ Preprocessed test data shape: {X_test.shape}")
print(f"✅ Training labels shape: {y_train_cat.shape}")
print(f"✅ Test labels shape: {y_test_cat.shape}")
print(f"✅ Pixel value range: {X_train.min():.1f} - {X_train.max():.1f}")

In [ ]:
# Data augmentation setup
print("🔄 Setting up data augmentation...")

datagen = ImageDataGenerator(
    rotation_range=10,      # Rotate images by up to 10 degrees
    width_shift_range=0.1,  # Shift images horizontally by up to 10%
    height_shift_range=0.1, # Shift images vertically by up to 10%
    zoom_range=0.1,         # Zoom in/out by up to 10%
    shear_range=0.1,        # Shear transformation
    fill_mode='nearest'     # Fill strategy for new pixels
)

# Fit the data generator
datagen.fit(X_train)

print("✅ Data augmentation configured!")

# Visualize augmented images
fig, axes = plt.subplots(2, 5, figsize=(15, 8))
fig.suptitle('🔄 Data Augmentation Examples', fontsize=16, fontweight='bold')

# Take one sample image
sample_image = X_train[0:1]  # Shape: (1, 28, 28, 1)

# Generate augmented versions
augmented_images = []
for batch in datagen.flow(sample_image, batch_size=1):
    augmented_images.append(batch[0])
    if len(augmented_images) >= 10:
        break

for i in range(10):
    row, col = i // 5, i % 5
    axes[row, col].imshow(augmented_images[i].squeeze(), cmap='gray')
    axes[row, col].set_title(f'Augmented {i+1}', fontweight='bold')
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

## 4. CNN Model Architecture

In [ ]:
# Build CNN model
print("🏗️ Building CNN architecture...")

model = keras.Sequential([
    # First Convolutional Block
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    
    # Second Convolutional Block
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    
    # Third Convolutional Block
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
    
    # Flatten and Dense Layers
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')  # 10 classes for digits 0-9
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ CNN model built and compiled!")
print("\n📋 Model Architecture:")
model.summary()

In [ ]:
# Visualize model architecture
print("🎨 Model Architecture Visualization:")

# Create a simple text representation
print("""
🧠 CNN Architecture Flow:

Input (28×28×1) 
    ↓
Conv2D (32 filters, 3×3) + ReLU + BatchNorm
    ↓
Conv2D (32 filters, 3×3) + ReLU
    ↓
MaxPooling (2×2) + Dropout (0.25)
    ↓
Conv2D (64 filters, 3×3) + ReLU + BatchNorm
    ↓
Conv2D (64 filters, 3×3) + ReLU
    ↓
MaxPooling (2×2) + Dropout (0.25)
    ↓
Conv2D (128 filters, 3×3) + ReLU + BatchNorm + Dropout (0.25)
    ↓
Flatten
    ↓
Dense (512) + ReLU + BatchNorm + Dropout (0.5)
    ↓
Dense (10) + Softmax
    ↓
Output (10 classes)
""")

# Count parameters
total_params = model.count_params()
print(f"📊 Total Parameters: {total_params:,}")
print(f"📊 Model Size: ~{total_params * 4 / 1024 / 1024:.1f} MB (float32)")